# Datapreparation

In this Notebook the scraped Data gets restructured. Also some additional informations will be added.

## Imports and Setups


In [1]:
import re
import json
import configparser
from datetime import datetime
import pandas as pd
import numpy as np
import googlemaps
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator


config = configparser.ConfigParser()
config.read('../local.ini')
gmaps_key = config['googlemaps']['key'].strip()
gmaps = googlemaps.Client(key=gmaps_key)

# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Translation
translator = GoogleTranslator(source='auto', target='en')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Loading the scraped Data 


In [2]:
# Read JSON data into a variable
json_data = open('../data/scraped/appart_info.json', encoding='utf-8').read()
# Parse JSON into a Python dictionary
data = json.loads(json_data)
# Create DataFrame from the JSON data
scraped_data = pd.DataFrame.from_dict(data, orient='index')
scraped_data

,WG_Size,Total_People,w,m,d,Title,Size,Rent,Extra_Costs,Other_Costs,...,Baujahr 1974,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Baujahr 2022,Baujahr 1955,Baujahr 1972,Flatrate\n 50-100 Mbit/s,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Erdgas leicht,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Energieeffizienzklasse\n A+,Baujahr 1952,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Holzpellets,Baujahr 1964
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.11109349.html,2,1,1,0,0,WG Zimmer Köln Nippes,17m²,393€,80€,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Neustadt-Sued.11109335.html,2,1,1,0,0,schönes WG Zimmer in der Südstadt Köln von 01....,12m²,600€,n.a.,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11096157.html,2,1,0,1,0,möbliertes WG Zimmer zur Zwischenmiete,20m²,800€,0€,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11109295.html,5,2,2,0,0,28qm in 5er WG mit Garten,28m²,720€,100€,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Koeln.10159188.html,3,2,2,0,0,TEMPORARY -Quiet flat share room in old buildi...,12m²,500€,n.a.,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.10995482.html,2,0,0,0,0,Große Wohnung in Köln Ehrenfeld,60m²,850€,150€,0€,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Altstadt-Sued.10991060.html,4,3,2,1,0,befristets Schönes Wohnliches Zimmer,17m²,680€,50€,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Humboldt-Gremberg.10949184.html,2,1,0,1,0,"Gästezimmer für eine Person, Nähe Messe-Deutz,...",10m²,40€,0€,0€,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.11002379.html,3,2,2,0,0,Wg Zimmer in Köln-Ehrenfeld (Zwischenmiete 25....,23m²,490€,70€,n.a.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Combining data into new columns
In order to use the scraped data, it must be prepared. To do this, the scraped data, which was previously saved in a JSON, is read in and saved in a dataframe. This dataframe contains 1027 entries with a total of 235 columns. The entries in many columns are empty except for some individual pieces of information.
To reduce the number of columns, the entries are combined into new columns and the original columns are then deleted. As a result, 176 columns were combined into 14 new columns, so that the dataframe now contains 73 columns.

### Construction Year
The year of construction of an apartment can be specified directly in an icon or in the text of various icons. The following code was used to extract this:

In [3]:
# Creating a new combined_data Dataframe
combined_data = scraped_data.copy()

# Neue Spalte 'Baujahr' hinzufügen
if 'Construction_Year' not in combined_data.columns:
    combined_data['Construction_Year'] = np.nan


for col in combined_data.columns:
    # Construction_Year from an Icon
    if col.startswith("Baujahr"):
        split_col = col.split(' ')
        if len(split_col) >= 2:
            year = split_col[1]
            combined_data.loc[combined_data[col] == 1, 'Construction_Year'] = float(year)
    
    # Construction_Year from a icon about energieefficiency
    elif str('Baujahr') in col:
        # Regex-structure for a construction year between 1700 and 2024
        pattern = r'\b(17[0-9]{2}|20[0-2][0-9]|202[0-4])\b'

        # Extract the construction year from the column name
        year = re.findall(pattern, col)
        if year:
            combined_data.loc[combined_data[col] == 1, 'Construction_Year'] = float(year[0])

# Drop columns that start with 'Baujahr' and are not 'Baujahr'
columns_to_drop = [col for col in combined_data.columns if col.startswith("Baujahr")]
combined_data.drop(columns=columns_to_drop, inplace=True)


### Consumption certificate (Verbrauchsausweis)

In [4]:
# Neue Spalte 'Verbrauchsausweis' hinzufügen
if 'Consumption_certificate' not in combined_data.columns:    
    combined_data['Consumption_certificate'] = ''

for col in combined_data.columns:
    if col.startswith("V:"):
        split_col = col.split(' ')
        if len(split_col) >= 3:
            value = split_col[1] + ' ' + split_col[2]
            combined_data.loc[combined_data[col] == 1, 'Consumption_certificate'] = str(value)

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if col.startswith("V:") ]
combined_data.drop(columns=columns_to_drop, inplace=True)


### WLAN Speed

In [5]:
if 'WLAN_speed_in_Mbit/s' not in combined_data.columns:   
    combined_data['WLAN_speed_in_Mbit/s'] = ''

for col in combined_data.columns:
    if str("Mbit/s") in col:
        col_name = col.replace('\n', '').strip()
        col_name = re.sub(r'\s+', ' ', col)
        split_col = col_name.split(' ')
        if len(split_col) >= 3:
            value = split_col[1:]
            value = ' '.join(value)
            combined_data.loc[combined_data[col] == 1, 'WLAN_speed_in_Mbit/s'] = str(value)

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if "Mbit/s" in col and col != 'WLAN_speed_in_Mbit/s']
combined_data.drop(columns=columns_to_drop, inplace=True)

### Distance to public transport 

In [6]:
if 'Dist_to_pub_trans_in_min' not in combined_data.columns:   
    combined_data['Dist_to_pub_trans_in_min'] = ''

for col in combined_data.columns:
    if str("zu Fuß entfernt") in col:
        col_name = col.replace('\n', '').strip()
        col_name = re.sub(r'\s+', ' ', col)
        split_col = col_name.split(' ')
        if len(split_col) >= 3:
            value = split_col[0]
            value = ' '.join(value)
            combined_data.loc[combined_data[col] == 1, 'Dist_to_pub_trans_in_min'] = str(value)

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if "zu Fuß entfernt" in col]
combined_data.drop(columns=columns_to_drop, inplace=True)


### Floor

In [7]:
# Neue Spalte 'Verbrauchsausweis' hinzufügen
if 'Floor' not in combined_data.columns:    
    combined_data['Floor'] = ''

# List of keywords for the floor
floor_keywords = ['OG', 'EG', 'UG', 'Hochparterre', 'Dachgeschoss', 'Souterrain', 'Penthouse', 'Tiefparterre']

for col in combined_data.columns:
    if any(keyword in col for keyword in floor_keywords):
        combined_data.loc[combined_data[col] == 1, 'Floor'] = col

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in floor_keywords)]
combined_data.drop(columns=columns_to_drop, inplace=True)

combined_data['Floor'] = combined_data['Floor'].str.replace('. OG', '').str.replace('Hochparterre', '0').str.replace('Tiefparterre', '0').replace('', np.nan).str.replace('EG', '0')




### Flooring 

In [8]:
# Prüfen und Spalte 'Bodenbelag' hinzufügen, falls nicht vorhanden
if 'Flooring' not in combined_data.columns:
    combined_data['Flooring'] = ''

# Liste der Bodenbelagsarten
bodenbelag_keywords = ['Teppich', 'Fliese', 'Parkett', 'Laminat', 'Stein', 'Kunststoff', 'Teppichboden', 
                       'Dielen', 'Kork', 'Marmor', 'Granit', 'Terrakotta', 'Beton', 'PVC']

# Überprüfen und Zuweisen
for col in combined_data.columns:
    if any(keyword in col for keyword in bodenbelag_keywords):
        combined_data.loc[combined_data[col] == 1, 'Flooring'] = col

# Spalten löschen
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in bodenbelag_keywords)]
combined_data.drop(columns=columns_to_drop, inplace=True)


### Parking

In [9]:
if 'Parking' not in combined_data.columns:    
    combined_data['Parking'] = ''

# Liste der Parkmöglichkeiten
Parking_keywords = ['Parkmöglichkeiten', 'Bewohnerparken', 'Garage', 'Carport', 'Parkhaus', 
                   'Tiefgarage', 'Duplex', 'Freiplatz', 'Parkplatz', 'Anwohnerparken', 'Parken', 'Stellplatz']

# Überprüfen und Zuweisen
for col in combined_data.columns:
    if any(keyword in col for keyword in Parking_keywords):
        combined_data.loc[combined_data[col] == 1, 'Parking'] = col

# Spalten löschen
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in Parking_keywords)]
combined_data.drop(columns=columns_to_drop, inplace=True)


### Energieefficiency

In [10]:
if 'Energieefficiency' not in combined_data.columns:    
    combined_data['Energieefficiency'] = ''

for col in combined_data.columns:
    if 'Energieeffizienz' in col:
        col_name = col.replace('\n', '').strip()
        col_name = re.sub(r'\s+', ' ', col)
        split_col = col_name.split(' ')
        combined_data.loc[combined_data[col] == 1, 'Energieefficiency'] = col
        if len(split_col) >= 2:
            value = split_col[-1]
            combined_data.loc[combined_data[col] == 1, 'Energieefficiency'] = str(value)
    if 'kein Energieausweis' in col:
        combined_data.loc[combined_data[col] == 1, 'Energieefficiency'] = 'kein Energieausweis nötig'

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if ('Energieeffizienz' in col and col != "Energieefficiency") or 'kein Energieausweis' in col]
combined_data.drop(columns=columns_to_drop, inplace=True)

### Garden/Balcony

In [11]:
if 'Garden_Balcony' not in combined_data.columns:    
    combined_data['Garden_Balcony'] = ''

# Liste der Garten/Balkon-Begriffe
garden_balcony_keywords = ['garten', 'Balkon', 'Garten', 'Terrasse', 'Loggia', 'Dachterrasse', 
                          'Wintergarten', 'Patio', 'Balkon/Terrasse', 'Gartenanteil', 'Balkonmöbel', 
                          'Balkonanzahl', 'Terrassenanzahl', 'Loggiaanzahl', 'Dachterrasseanzahl', 
                          'Wintergartenanzahl', 'Patioanzahl']

# Überprüfen und Zuweisen
for col in combined_data.columns:
    if any(keyword in col for keyword in garden_balcony_keywords):
        combined_data.loc[combined_data[col] == 1, 'Garden_Balcony'] = col

# Spalten löschen
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in garden_balcony_keywords) and col != 'Garden_Balcony']
combined_data.drop(columns=columns_to_drop, inplace=True)


### Basement/Cellar

In [12]:
if 'Cellar' not in combined_data.columns:    
    combined_data['Cellar'] = ''

for col in combined_data.columns:
    if 'Keller' in col or 'Fahrradkeller' in col:
        combined_data.loc[combined_data[col] == 1, 'Cellar'] = col

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if ('Keller' in col or 'Fahrradkeller' in col) and col != 'Cellar']
combined_data.drop(columns=columns_to_drop, inplace=True)

### Heating

In [13]:
if 'Heating' not in combined_data.columns:    
    combined_data['Heating'] = ''

heating_keywords = ['Erdgas', 'heizung', 'Fernwärme', 'Gas', 'Öl', 'Nachtspeicher', 'Luftwärmepumpe']

for col in combined_data.columns:
    if any(keyword in col for keyword in heating_keywords):
        combined_data.loc[combined_data[col] == 1, 'Heating'] = col

# Drop columns 
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in heating_keywords)]
combined_data.drop(columns=columns_to_drop, inplace=True)


### Construction Type

In [14]:
if 'Constructing_Type' not in combined_data.columns:    
    combined_data['Constructing_Type'] = ''

bautyp_keywords = ['Altbau', 'Neubau', 'Hochhaus', 'Mehrfamilienhaus', 'Reihenhaus', 'Doppelhaus', 'Plattenbau', 'Einfamilienhaus']

for col in combined_data.columns:
    for keyword in bautyp_keywords:
        if keyword in col:
            combined_data.loc[combined_data[col] == 1, 'Constructing_Type'] = combined_data.apply(
                lambda row: (row['Constructing_Type'] + ', ' + col).strip(', ') if row['Constructing_Type'] != '' else col,
                axis=1
            )

# Removing leading commas
combined_data['Constructing_Type'] = combined_data['Constructing_Type'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))


# Drop columns 
columns_to_drop = [col for col in combined_data.columns if any(keyword in col for keyword in bautyp_keywords)]
combined_data.drop(columns=columns_to_drop, inplace=True)



### Smoking

In [15]:
# Combining "Rauchen im Zimmer erlaubt" and "Rauchen auf dem Balkon erlaubt" into one category
combined_data['Smoking'] = combined_data['Rauchen'].str.replace('Rauchen im Zimmer erlaubt', 'Rauchen teilweise erlaubt').str.replace('Rauchen auf dem Balkon erlaubt', 'Rauchen teilweise erlaubt')



### Age Restrictions

In [16]:
def extract_min_max_age(row):
    # Extrahieren der Altersinformationen aus beiden Spalten
    ages_col1 = [int(age) for age in re.findall(r'\d+', str(row['Search_Info']))]
    ages_col2 = [int(age) for age in re.findall(r'\d+', str(row['Alter']))]
    
    # Kombinieren der Listen und Entfernen von leeren Werten
    all_ages = ages_col1 + ages_col2
    all_ages = [age for age in all_ages if age != []]
    
    # Rückgabe des Mindestalters, falls vorhanden, sonst NaN
    return (min(all_ages), max(all_ages)) if all_ages else np.nan
combined_data[['Min_Age', 'Max_Age']] = combined_data.apply(extract_min_max_age, axis=1).apply(pd.Series)



### Checking the DF

In [17]:
pd.set_option('display.max_columns', 100)
combined_data

,WG_Size,Total_People,w,m,d,Title,Size,Rent,Extra_Costs,Other_Costs,Deposit,Redemption_Agreement,Address,Available_From,Available_Till,Online_Since,Search_Info,Ökostrom,Bewerbermappe,itsmydata,SCHUFA,Ausweis/ID,Einkommensnachweis,Mieterselbstauskunft,Wohnungsgröße,Alter,Rauchen,Zweck-WG,gemischte-WG,Sprache/n,Zimmer,Lage,WG-Leben,Sonstiges,Dusche,Waschmaschine,möbliert,Spülmaschine,WLAN,Kabel,Frauen-WG,Studenten-WG,Badewanne,Mietzahlungsbestätigung,Aufzug,teilmöbliert,Berufstätigen-WG,DSL,Flatrate,Männer-WG,Verbindung,LGBTQ,Satellit,Azubi-WG,Barrierefrei,Bürgschaft,Haustiere erlaubt,Strom,Construction_Year,Consumption_certificate,WLAN_speed_in_Mbit/s,Dist_to_pub_trans_in_min,Floor,Flooring,Parking,Energieefficiency,Garden_Balcony,Cellar,Heating,Constructing_Type,Smoking,Min_Age,Max_Age
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.11109349.html,2,1,1,0,0,WG Zimmer Köln Nippes,17m²,393€,80€,n.a.,982€,n.a.,Merheimer Straße 332 50733 Köln Nippes,01.08.2024,01.10.2024,1 Stunde,Geschlecht egal zwischen 20 und 30 Jahren,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Wohnungsgröße: 55m²,Bewohneralter: 23 bis 25 Jahre,Rauchen nicht erwünscht,0.0,1.0,"Sprache/n: Deutsch, Englisch",Das ganze Haus ist ein Altbau und dementsprech...,Die Lage ist perfekt für Leute die alles gut e...,"Das WG-Leben ist schön, meine liebe Mitbewohne...",Es gibt einen Kellerraum für Fahrräder und ein...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,7,2,,Bewohnerparken,,,,,Altbau,Rauchen nicht erwünscht,20.0,30.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Neustadt-Sued.11109335.html,2,1,1,0,0,schönes WG Zimmer in der Südstadt Köln von 01....,12m²,600€,n.a.,n.a.,900€,n.a.,Bonnerstr.2 50677 Köln Neustadt-Süd,01.08.2024,30.09.2024,1 Stunde,Frau bis 30 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Rauchen nicht erwünscht,NaN,NaN,"Sprache/n: Deutsch, Englisch",Ich biete ein schönes 12 qm großes WG-Zimmer i...,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,0,Parkett,,,Balkon,,,sanierter Altbau,Rauchen nicht erwünscht,30.0,30.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11096157.html,2,1,0,1,0,möbliertes WG Zimmer zur Zwischenmiete,20m²,800€,0€,n.a.,500€,n.a.,Hans-Sachs-Straße 1 50931 Köln Lindenthal,01.08.2024,01.09.2024,1 Stunde,Geschlecht egal zwischen 20 und 30 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,Wohnungsgröße: 84m²,Bewohneralter: 25 bis 26 Jahre,Rauchen auf dem Balkon erlaubt,NaN,NaN,"Sprache/n: Deutsch, Englisch","Da wir für einen Monat auf Reisen sind, suchen...",NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2,1,,Bewohnerparken,,Balkon,,,sanierter Altbau,Rauchen teilweise erlaubt,20.0,30.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11109295.html,5,2,2,0,0,28qm in 5er WG mit Garten,28m²,720€,100€,n.a.,1440€,n.a.,Werthmannstraße 17 50935 Köln Lindenthal,15.07.2024,n.a.,1 Stunde,Geschlecht egal zwischen 25 und 35 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,Bewohneralter: 27 bis 27 Jahre,NaN,0.0,1.0,"Sprache/n: Deutsch, Englisch","Das Zimmer ist Teil einer 5er WG, die Ende Jul...",Die Lage ist wirklich ein Traum! Deckstein is...,Canan und ich sind schon seit 8 Jahren befreun...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,5,0,,,,Garten,Keller,Gasheizung,,NaN,25.0,35.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Koeln.10159188.html,3,2,2,0,0,TEMPORARY -Quiet flat share room in old buildi...,12m²,500€,n.a.,n.a.,250€,n.a.,Herthastraße 60 50969 Köln Köln,22.07.2024,31.08.2024,1 Stunde,Geschlecht egal,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,Bewohneralter: ab 25 Jahre,NaN,NaN,1.0,"Sprache/n: Deutsch, Englisch, Spanisch, Franzö...","Hello, I'm Axelle, I'm 25 and I'm studying f...","Sunny street, quiet, you can find everything a...",There are 4 of us living in this beautiful apa...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,

## Removing Units from cells
To use the information given in the dataframe we need float or int values. Therefor we remove the
units from the cells and save them in the column name. 

In [18]:
combined_data['Cold_Rent_in_€'] = combined_data['Rent'].str.replace('€', '').replace('n.a.', np.nan)
combined_data['Extra_Costs_in_€'] = combined_data['Extra_Costs'].str.replace('€', '').replace('n.a.', np.nan)
combined_data['Other_Costs_in_€'] = combined_data['Other_Costs'].str.replace('€', '').replace('n.a.', np.nan)
combined_data['Deposit_in_€'] = combined_data['Deposit'].str.replace('€', '').replace('n.a.', np.nan)
combined_data['Roomsize in m²'] = combined_data['Size'].str.replace('m²', '').replace('n.a.', np.nan)
combined_data['Appartmentsize in m²'] = combined_data['Wohnungsgröße'].str.replace('Wohnungsgröße: ', '').str.replace('m²', '').replace('n.a.', np.nan)

## Adding Information

### Transform Online Since
The duration of how long an ad has been online is specified in various units and formats. If an ad was created less than an hour ago, the duration is given in minutes. This is followed by an indication first in hours and then in ays. If an ad has been online for a long time, the date on which it was created is displayed. In order to bring this to a uniform scale, the time was converted to hours for all cases. 


In [19]:
def convert_to_hours(entry):
        if 'Stunde' in entry:
            return int(entry.split()[0])
        elif 'Minuten' in entry:
            return 1
        elif 'Tag' in entry:
            return int(entry.split()[0]) * 24
        else:
            # Annahme: Datum im Format 'DD.MM.YYYY'
            given_date = datetime.strptime(entry, '%d.%m.%Y')
            reference_date = datetime(2024, 7, 9)  # Referenzdatum 09.07.2024
            hours_difference = (reference_date - given_date).total_seconds() / 3600
            return int(hours_difference)
combined_data['Online_since_in_h'] = combined_data['Online_Since'].apply(convert_to_hours)

### Count named Facts
The number of entries is counted here. 11 details exist in all advertisements and are not included in order not to distort the result of the coefficient in a possible regression 

In [20]:
combined_data['Fact_Count'] = combined_data.apply(lambda row: sum((row.notna()) & (row != '')) - 11, axis=1)

### Check for intermediate rent
Here it is checked whether the room is only to be rented temporarily

In [21]:
combined_data['Intermediate Rent'] = combined_data['Available_Till'].apply(lambda x: True if x!='n.a.' else False)

### Calculate Warm Rent

In [22]:
combined_data['Warm_Rent_in_€'] = combined_data['Cold_Rent_in_€'].astype(float) + combined_data['Extra_Costs_in_€'].fillna(0).astype(float) 

### Calculate Warm Rent per m²

In [23]:
combined_data['Price_per_m²'] = combined_data['Warm_Rent_in_€'].astype(float) / combined_data['Roomsize in m²'].astype(float)

### Titlelength

In [24]:
combined_data['Title_Length'] = combined_data['Title'].str.len()

### Descriptionlength
The length of the 4 possible descriptions is added together to estimate the total length of the
description. 

In [25]:
combined_data['Description_Length'] = combined_data['Zimmer'].str.len().fillna(0) + combined_data['Lage'].str.len().fillna(0) + combined_data['WG-Leben'].str.len().fillna(0) + combined_data['Sonstiges'].str.len().fillna(0)

### Postal Code related informations
Some information could be related to the respective district or postal code. An attempt is therefore made to add information to the data set via the zip code. In particular, information about the Rhine side and the population density. The right bank of the Rhine is sometimes referred to by Cologne residents as the "wrong bank of the Rhine". There could therefore be price differences. 

#### Extract Postal Codes

In [26]:
postal_codes = combined_data[['Address']].copy()

plz_pattern = re.compile(r'\b\d{5}\b')
postal_codes['plz'] = postal_codes['Address'].apply(lambda x: plz_pattern.search(x).group(0) if plz_pattern.search(x) else np.nan)

#### Get information about Population per km² in cologne

In [27]:
plz_de = pd.read_excel('../data/plz_einwohner.xlsx')
plz_cologne = plz_de[plz_de['note'].str.contains('Köln')].copy()
plz_cologne['Population_per_km²'] = (plz_cologne['einwohner'] / plz_cologne['qkm']).round(2)
plz_cologne.drop(columns=['note', 'einwohner', 'qkm', 'lat', 'lon'], inplace=True)

#### Add information about the Rhine-Side

In [28]:
data = {
    "plz": [50667, 50668, 50670, 50672, 50674, 50676, 50677, 50678, 50679, 50733, 50735, 50737, 50739, 50765, 50767, 50769, 50823, 50825, 50827, 50829, 50858, 50859, 50931, 50933, 50935, 50937, 50939, 50968, 50969, 50996, 50997, 50999, 51061, 51063, 51065, 51067, 51069, 51103, 51105, 51107, 51109, 51143, 51145, 51147, 51149],
    "left_rhine_side": [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

# DataFrame erstellen
rhine_side = pd.DataFrame(data)

#### Merging the postal code related informations

In [29]:
plz_cologne['plz'] = pd.to_numeric(plz_cologne['plz'], errors='coerce')
rhine_side['plz'] = pd.to_numeric(rhine_side['plz'], errors='coerce')
postal_code_data =pd.merge(plz_cologne, rhine_side, on='plz', how='left')


postal_codes['plz'] = pd.to_numeric(postal_codes['plz'], errors='coerce')
postal_code_data['plz'] = pd.to_numeric(postal_code_data['plz'], errors='coerce')
postal_codes_res = pd.merge(postal_codes, postal_code_data, on='plz', how='left')

# Keep the Index of combined_data
postal_codes_res.index = combined_data.index
postal_codes_res.drop(columns=['Address'], inplace=True)
postal_codes_res

,plz,Population_per_km²,left_rhine_side
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.11109349.html,50733.0,11259.96,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Neustadt-Sued.11109335.html,50677.0,12150.68,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11096157.html,50931.0,6467.99,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11109295.html,50935.0,3283.34,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Koeln.10159188.html,50969.0,5048.95,1.0
...,...,...,...
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.10995482.html,50823.0,11677.08,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Altstadt-Sued.10991060.html,50674.0,12311.83,1.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Humboldt-Gremberg.10949184.html,51105.0,3446.98,0.0
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.11002379.html,50825.0,7241.29,1.0


#### Join to the large Dataframe

In [30]:
# Identify overlapping columns
overlapping_columns = combined_data.columns.intersection(postal_codes_res.columns)

# Drop these columns from the right DataFrame
postal_codes_res_filtered = postal_codes_res.drop(columns=overlapping_columns)

# Perform the join operation on the index
combined_data = combined_data.join(postal_codes_res_filtered)

### Near to City Centre
Assumptions: 
- Cologne cathedral is the city centre
- relevant distance is the walking distance

In [31]:
def calc_dist(start):
    return gmaps.directions(start, "Domkloster 4, 50667 Köln", mode="walking", departure_time=datetime.now())[0]['legs'][0]['distance']['text']

combined_data['Dist_center_in_km'] = combined_data['Address'].apply(calc_dist)
combined_data['Dist_center_in_km'] = combined_data['Dist_center_in_km'].str.replace(' km', '').astype(float)


### Sentiment Analysis

The sentiment of texts can be determined relatively easily with the help of the nltk library. However, this is primarily designed for English texts. Therefore, the text is first translated into English before a
sentiment is calculated. The sentiment score is calculated from the weighted sentiments of the individual descriptions. The weighting is based on the length of the respective texts.

#### Reducing the length of the descriptions

In [32]:
def limit_text(text, max_chars=4999):
    if len(text) > max_chars:
        return text[:max_chars]
    else:
        return text
    
combined_data['Zimmer'] = combined_data['Zimmer'].apply(str).apply(limit_text)          # 2 descriptions are to long
combined_data['Lage'] = combined_data['Lage'].apply(str).apply(limit_text)              # 0 descriptions are to long
combined_data['WG-Leben'] = combined_data['WG-Leben'].apply(str).apply(limit_text)      # 2 descriptions are to long
combined_data['Sonstiges'] = combined_data['Sonstiges'].apply(str).apply(limit_text)    # 0 descriptions are to long

#### Translating

In [33]:
def translate(text):
    if text == 'nan':
        return ''
    else:
        return translator.translate(text)



combined_data['Zimmer_engl'] = combined_data['Zimmer'].apply(str).apply(translate)
combined_data['Lage_engl'] = combined_data['Lage'].apply(str).apply(translate)
combined_data['WG-Leben_engl'] = combined_data['WG-Leben'].apply(str).apply(translate)
combined_data['Sonstiges_engl'] = combined_data['Sonstiges'].apply(str).apply(translate)


#### Calculating Sentiment and Weighted_Average_Sentiment

In [34]:
# TODO: change test to combined_data 
def analyze_sentiment(description):
    # Ensure description is a string
    if not isinstance(description, str):
        description = str(description)
    sentiment = sia.polarity_scores(description)
    return sentiment['compound']


combined_data['Weighted_Average_Sentiment'] = ( (combined_data['Zimmer_engl'].apply(analyze_sentiment) * combined_data['Zimmer_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['Lage_engl'].apply(analyze_sentiment) * combined_data['Lage_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['WG-Leben_engl'].apply(analyze_sentiment) * combined_data['WG-Leben_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['Sonstiges_engl'].apply(analyze_sentiment) * combined_data['Sonstiges_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0)) 
                                        /
                                        (combined_data['Zimmer_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['Lage_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['WG-Leben_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0) +
                                        combined_data['Sonstiges_engl'].apply(lambda x: len(x) if isinstance(x, str) else 0))
                                        )




In [35]:
combined_data

,WG_Size,Total_People,w,m,d,Title,Size,Rent,Extra_Costs,Other_Costs,Deposit,Redemption_Agreement,Address,Available_From,Available_Till,Online_Since,Search_Info,Ökostrom,Bewerbermappe,itsmydata,SCHUFA,Ausweis/ID,Einkommensnachweis,Mieterselbstauskunft,Wohnungsgröße,Alter,Rauchen,Zweck-WG,gemischte-WG,Sprache/n,Zimmer,Lage,WG-Leben,Sonstiges,Dusche,Waschmaschine,möbliert,Spülmaschine,WLAN,Kabel,Frauen-WG,Studenten-WG,Badewanne,Mietzahlungsbestätigung,Aufzug,teilmöbliert,Berufstätigen-WG,DSL,Flatrate,Männer-WG,Verbindung,LGBTQ,Satellit,Azubi-WG,Barrierefrei,Bürgschaft,Haustiere erlaubt,Strom,Construction_Year,Consumption_certificate,WLAN_speed_in_Mbit/s,Dist_to_pub_trans_in_min,Floor,Flooring,Parking,Energieefficiency,Garden_Balcony,Cellar,Heating,Constructing_Type,Smoking,Min_Age,Max_Age,Cold_Rent_in_€,Extra_Costs_in_€,Other_Costs_in_€,Deposit_in_€,Roomsize in m²,Appartmentsize in m²,Online_since_in_h,Fact_Count,Intermediate Rent,Warm_Rent_in_€,Price_per_m²,Title_Length,Description_Length,plz,Population_per_km²,left_rhine_side,Dist_center_in_km,Zimmer_engl,Lage_engl,WG-Leben_engl,Sonstiges_engl,Weighted_Average_Sentiment
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.11109349.html,2,1,1,0,0,WG Zimmer Köln Nippes,17m²,393€,80€,n.a.,982€,n.a.,Merheimer Straße 332 50733 Köln Nippes,01.08.2024,01.10.2024,1 Stunde,Geschlecht egal zwischen 20 und 30 Jahren,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Wohnungsgröße: 55m²,Bewohneralter: 23 bis 25 Jahre,Rauchen nicht erwünscht,0.0,1.0,"Sprache/n: Deutsch, Englisch",Das ganze Haus ist ein Altbau und dementsprech...,Die Lage ist perfekt für Leute die alles gut e...,"Das WG-Leben ist schön, meine liebe Mitbewohne...",Es gibt einen Kellerraum für Fahrräder und ein...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,7,2,,Bewohnerparken,,,,,Altbau,Rauchen nicht erwünscht,20.0,30.0,393,80,NaN,982,17,55,1,36,True,473.0,27.823529,21,1179.0,50733.0,11259.96,1.0,3.7,The whole house is an old building and the roo...,The location is perfect for people who want to...,"Life in a shared apartment is nice, my lovely ...",There is a basement room for bicycles and a st...,0.828058
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Neustadt-Sued.11109335.html,2,1,1,0,0,schönes WG Zimmer in der Südstadt Köln von 01....,12m²,600€,n.a.,n.a.,900€,n.a.,Bonnerstr.2 50677 Köln Neustadt-Süd,01.08.2024,30.09.2024,1 Stunde,Frau bis 30 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Rauchen nicht erwünscht,NaN,NaN,"Sprache/n: Deutsch, Englisch",Ich biete ein schönes 12 qm großes WG-Zimmer i...,nan,nan,nan,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,0,Parkett,,,Balkon,,,sanierter Altbau,Rauchen nicht erwünscht,30.0,30.0,600,NaN,NaN,900,12,NaN,1,31,True,600.0,50.000000,58,1443.0,50677.0,12150.68,1.0,2.7,I am offering a beautiful 12 sqm shared room i...,,,,0.993400
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11096157.html,2,1,0,1,0,möbliertes WG Zimmer zur Zwischenmiete,20m²,800€,0€,n.a.,500€,n.a.,Hans-Sachs-Straße 1 50931 Köln Lindenthal,01.08.2024,01.09.2024,1 Stunde,Geschlecht egal zwischen 20 und 30 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,Wohnungsgröße: 84m²,Bewohneralter: 25 bis 26 Jahre,Rauchen auf dem Balkon erlaubt,NaN,NaN,"Sprache/n: Deutsch, Englisch","Da wir für einen Monat auf Reisen sind, suchen...",nan,nan,nan,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2,1,,Bewohnerparken,,Balkon,,,sanierter Altbau,Rauchen teilweise erlaubt,20.0,30.0,800,0,NaN,500,20,84,1,30,True,800.0,40.000000,38,505.0,50931.0,6467.99,1.0,3.4,"Since we are traveling for a month, we are loo...",,,,0.599400
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11109295.html,5,2,2,0,0,28qm in 5er WG mit Garten,28m²,720€,100€,n.a.,1440€,n.a.,Werthmannstraße 17 50935 Köln Lindenthal,15.07.2024,n.a.,1 Stunde,Geschlecht egal zwischen 25 und 35 Jahren,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,Bewohnera

## Picking final columns
Before the data record is exported, relevant columns are selected. 


In [37]:
final_df = combined_data[['Price_per_m²', 'Online_since_in_h', 'WG_Size', 'Total_People', 'Size', 'Warm_Rent_in_€', 'Cold_Rent_in_€', 'Deposit_in_€', 'Roomsize in m²', 'Appartmentsize in m²', 'Intermediate Rent', 'left_rhine_side', 'Weighted_Average_Sentiment', 'Min_Age', 'Max_Age', 'Construction_Year', 'Parking', 'Floor', 'Smoking', 'Garden_Balcony', 'Dist_center_in_km', 'Title_Length', 'Description_Length', 'Fact_Count', 'Population_per_km²']]
pd.set_option('display.max_columns', 100)
final_df


,Price_per_m²,Online_since_in_h,WG_Size,Total_People,Size,Warm_Rent_in_€,Cold_Rent_in_€,Deposit_in_€,Roomsize in m²,Appartmentsize in m²,Intermediate Rent,left_rhine_side,Weighted_Average_Sentiment,Min_Age,Max_Age,Construction_Year,Parking,Floor,Smoking,Garden_Balcony,Dist_center_in_km,Title_Length,Description_Length,Fact_Count,Population_per_km²
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.11109349.html,27.823529,1,2,1,17m²,473.0,393,982,17,55,True,1.0,0.828058,20.0,30.0,NaN,Bewohnerparken,2,Rauchen nicht erwünscht,,3.7,21,1179.0,36,11259.96
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Neustadt-Sued.11109335.html,50.000000,1,2,1,12m²,600.0,600,900,12,NaN,True,1.0,0.993400,30.0,30.0,NaN,,0,Rauchen nicht erwünscht,Balkon,2.7,58,1443.0,31,12150.68
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11096157.html,40.000000,1,2,1,20m²,800.0,800,500,20,84,True,1.0,0.599400,20.0,30.0,NaN,Bewohnerparken,1,Rauchen teilweise erlaubt,Balkon,3.4,38,505.0,30,6467.99
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Lindenthal.11109295.html,29.285714,1,5,2,28m²,820.0,720,1440,28,NaN,False,1.0,0.945505,25.0,35.0,NaN,,0,NaN,Garten,5.4,25,2568.0,28,3283.34
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Koeln.10159188.html,41.666667,1,3,2,12m²,500.0,500,250,12,NaN,True,1.0,0.803016,25.0,25.0,NaN,gute Parkmöglichkeiten,1,NaN,Balkon,4.0,61,1421.0,34,5048.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.10995482.html,16.666667,1344,2,0,60m²,1000.0,850,250,60,NaN,True,1.0,0.856763,NaN,NaN,NaN,gute Parkmöglichkeiten,2,NaN,Balkon,3.2,31,1642.0,33,11677.08
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Altstadt-Sued.10991060.html,42.941176,1344,4,3,17m²,730.0,680,200,17,70,True,1.0,0.963600,20.0,27.0,NaN,,1,Rauchen teilweise erlaubt,,2.7,36,1402.0,28,12311.83
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Humboldt-Gremberg.10949184.html,4.000000,1368,2,1,10m²,40.0,40,0,10,47,True,0.0,0.951300,18.0,99.0,1964.0,gute Parkmöglichkeiten,0,Rauchen nicht erwünscht,,3.6,74,766.0,35,3446.98
https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Ehrenfeld.11002379.html,24.347826,1368,3,2,23m²,560.0,490,NaN,23,NaN,True,1.0,0.980600,21.0,23.0,NaN,,2,Rauchen teilweise erlaubt,Balkon,3.4,55,641.0,24,7241.29


### Exporting

In [38]:
final_df.to_csv('../data/final_data.csv', index=False)